In [ ]:
!pip3 install python-rake keybert

In [125]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from math import floor
import RAKE
from keybert import KeyBERT
from nltk.tag import pos_tag

punctuation += '«»...“’‘'
lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Датасет

Рандомный поддатасет из большого с собранного по 5 новостным источникам с октября по ноябрь 2017

In [93]:
data = pd.read_csv('/content/drive/MyDrive/nytimes.csv')
data.head()

,Unnamed: 0,KEYWORDS,SUMMARY,TEXT,TITLE,URL
0,0,"['swaggering', 'sex', 'aggs', 'shes', 'feminis...",PhotoI HEAR SHE’S A REAL BITCHBy Jen AggIllust...,Photo\n\nI HEAR SHE’S A REAL BITCH\n\nBy Jen A...,"‘I Hear She’s a Real Bitch’: A Swaggering, Fem...",https://www.nytimes.com/2017/11/01/books/revie...
1,1,"['russians', 'case', 'papadopoulos', 'trump', ...",What else about Trump’s obsequious overtures t...,“Well before any public knowledge of these eve...,The Sleazy Case Against Mueller’s Probe,https://www.nytimes.com/2017/11/03/opinion/rob...
2,2,"['international', 'antagonistic', 'myanmar', '...",Advertisement Continue reading the main storyT...,"Further complicating things, analysts said, wa...",Fate of Stateless Rohingya Muslims Is in Antag...,https://www.nytimes.com/2017/11/03/world/asia/...
3,3,"['war', 'shortos', 'shorto', 'eyes', 'sets', '...",Both Cornplanter and Margaret Coghlan met Geor...,Another of Shorto’s subjects is Margaret Moncr...,"The War of Independence, Seen Through Six Sets...",https://www.nytimes.com/2017/10/30/books/revie...
4,4,"['thats', 'tough', 'wilber', 'donaldson', 'rog...",AdvertisementWordplay Exquisite Gem Image A va...,Advertisement\n\nWordplay Exquisite Gem Image ...,Exquisite Gem,https://www.nytimes.com/2017/10/20/crosswords/...


Объединим заголовок и новость

In [94]:
data['FULL_TEXT'] = data['TITLE'] + data['TEXT']

Посчитаем сколько word-токенов в 1 новости

In [95]:
data[:1]['FULL_TEXT'].apply(lambda x: len([token for token in word_tokenize(x, language = 'english') if token not in punctuation]))[0]

1016

Возьмем корпус из 5 новостей и выделим в нем ключевые слова вручную, заодно посчитаем общую длину датасета

In [96]:
data = data[:5]

cnt = 0
manual_keywords = []

for text in data['FULL_TEXT']:
  cnt += len([token for token in word_tokenize(text, language = 'english')])
  print(text)
  print('-------')
  keywords = input()
  manual_keywords.append(keywords.split(', '))

data['MAN_KEYWORDS'] = manual_keywords
cnt

‘I Hear She’s a Real Bitch’: A Swaggering, Feminist Restaurant MemoirPhoto

I HEAR SHE’S A REAL BITCH

By Jen Agg

Illustrated. 355 pp. Penguin Books. Paper, $17.

From the first pages of Jen Agg’s memoir, “I Hear She’s a Real Bitch,” a reader knows she is dealing with a formidable ego. Agg isn’t a writer but a restaurateur. She owns five beloved restaurants and bars in Canada, where she is a fixture of the food press, both for her success and for her occasionally impolitic forthrightness. In “I Hear She’s a Real Bitch,” Agg brags, she swaggers, she writes in great detail about her appetites — for junk food, music, sex. She is also relentlessly digressive — one of the truest signs of robust self-regard. Reading Agg’s book, I felt as if I was spending hours with someone who was pacing and rambling, expecting me to scribble disorderly dictation into my steno pad, stopping on occasion to ask me imperiously: “Are you getting all this?”

Whether I was depended entirely on which of the sever

4556

In [97]:
data = data.drop(columns = ['Unnamed: 0', 'SUMMARY', 'TEXT', 'TITLE', 'URL'], axis = 1)
data

,KEYWORDS,FULL_TEXT,MAN_KEYWORDS
0,"['swaggering', 'sex', 'aggs', 'shes', 'feminis...","‘I Hear She’s a Real Bitch’: A Swaggering, Fem...","[real, bitch, feminist, memoirs, book]"
1,"['russians', 'case', 'papadopoulos', 'trump', ...",The Sleazy Case Against Mueller’s Probe“Well b...,"[Russian, russians, Kremlin, Mueller, Clinton,..."
2,"['international', 'antagonistic', 'myanmar', '...",Fate of Stateless Rohingya Muslims Is in Antag...,"[Rohingya, Bangladesh, refugees, violence]"
3,"['war', 'shortos', 'shorto', 'eyes', 'sets', '...","The War of Independence, Seen Through Six Sets...","[Shorto, war, revolution, death, Washington, N..."
4,"['thats', 'tough', 'wilber', 'donaldson', 'rog...",Exquisite GemAdvertisement\n\nWordplay Exquisi...,"[gem, puzzle, Saturday, Friday, play]"


Сравним ключевые слова, в оригинальных выделено больше и иногда не по смыслу, а по частотности. Объединим мою разметку и оригинальную как эталон

In [98]:
for orig, man in zip(data['KEYWORDS'], data['MAN_KEYWORDS']):
  print(orig)
  print(man)
  print('---')

['swaggering', 'sex', 'aggs', 'shes', 'feminist', 'agg', 'men', 'memoir', 'real', 'restaurant', 'bitch', 'book', 'hear']
['real', 'bitch', 'feminist', 'memoirs', 'book']
---
['russians', 'case', 'papadopoulos', 'trump', 'clinton', 'russian', 'times', 'sleazy', 'muellers', 'sign', 'probe', 'manafort', 'campaign', 'steele']
['Russian', 'russians', 'Kremlin', 'Mueller', 'Clinton', 'Trump', 'Manafort', 'campaign']
---
['international', 'antagonistic', 'myanmar', 'fate', 'rohingya', 'refugees', 'york', 'main', 'including', 'muslims', 'continue', 'reading', 'stateless', 'hands', 'group']
['Rohingya', 'Bangladesh', 'refugees', 'violence']
---
['war', 'shortos', 'shorto', 'eyes', 'sets', 'revolution', 'washington', 'american', 'george', 'smith', 'coghlan', 'independence', 'yorks', 'york', 'seen']
['Shorto', 'war', 'revolution', 'death', 'Washington', 'New York', 'victory']
---
['thats', 'tough', 'wilber', 'donaldson', 'rogue', 'puzzle', 'started', '_', 'gem', 'exquisite', 'im', 'help']
['gem',

In [99]:
def list_lower(li):
  return [elem.lower() for elem in li]

data['MAN_KEYWORDS'].apply(list_lower)
data['KEYWORDS'] = data['KEYWORDS'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').split(', '))
data['KEYWORDS_FULL'] = [set(orig).union(set(man)) for orig, man in zip(data['KEYWORDS'].tolist(), data['MAN_KEYWORDS'].tolist())]

In [100]:
data = data.drop(columns = ['KEYWORDS', 'MAN_KEYWORDS'], axis = 1)
# и еще запрепроцессим статьи и оставим неотпроцешенные для берта и rake

def preprocess(text):
  text = word_tokenize(text, language = 'english')
  preprocessed_text =  [lemmatizer.lemmatize(token.lower()) for token in text
                        if token not in punctuation and token.lower() not in stop]
  return ' '.join(preprocessed_text)

data['FULL_TEXT_PROCESSED'] = data['FULL_TEXT'].apply(preprocess)
data

,FULL_TEXT,KEYWORDS_FULL,FULL_TEXT_PROCESSED
0,"‘I Hear She’s a Real Bitch’: A Swaggering, Fem...","{memoir, shes, sex, swaggering, feminist, men,...",hear real bitch swaggering feminist restaurant...
1,The Sleazy Case Against Mueller’s Probe“Well b...,"{sign, manafort, trump, times, sleazy, Kremlin...",sleazy case mueller probe well public knowledg...
2,Fate of Stateless Rohingya Muslims Is in Antag...,"{myanmar, stateless, including, Rohingya, musl...",fate stateless rohingya muslim antagonistic ha...
3,"The War of Independence, Seen Through Six Sets...","{revolution, death, eyes, smith, yorks, washin...",war independence seen six set eyesanother shor...
4,Exquisite GemAdvertisement\n\nWordplay Exquisi...,"{wilber, tough, puzzle, Friday, donaldson, hel...",exquisite gemadvertisement wordplay exquisite ...


### Методы выделения ключевых слов

##### TF-IDF

Часть взята [отсюда](https://www.kaggle.com/code/rowhitswami/keywords-extraction-using-tf-idf-method)

In [101]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(data['FULL_TEXT_PROCESSED'].tolist())
feature_names = vectorizer.get_feature_names()

TOP_N_KEYWORDS = floor(data['KEYWORDS_FULL'].apply(lambda x: len(x)).mean())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [102]:
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

def get_keywords(doc, vectorizer=vectorizer, feature_names=feature_names, TOP_K_KEYWORDS=TOP_N_KEYWORDS):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform([doc])
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)
    
    return list(keywords.keys())

In [103]:
data['TFIDF_KEYWORDS'] = data['FULL_TEXT_PROCESSED'].apply(get_keywords)

##### RAKE

In [110]:
rake = RAKE.Rake(stop)

def rake_for_df(text):
  # maxWords = 2, так как у нас в разметке максимум биграммы
  #minCharacters = 3 так как иногда попадает пунктуация так как тексты необработанные
  result = rake.run(text,  maxWords = 2, minFrequency = 2, minCharacters = 3)
  return [elem[0] for elem in result]

data['RAKE_KEYWORDS'] = data['FULL_TEXT'].apply(rake_for_df)

In [111]:
data

,FULL_TEXT,KEYWORDS_FULL,FULL_TEXT_PROCESSED,TFIDF_KEYWORDS,RAKE_KEYWORDS
0,"‘I Hear She’s a Real Bitch’: A Swaggering, Fem...","{memoir, shes, sex, swaggering, feminist, men,...",hear real bitch swaggering feminist restaurant...,"[agg, restaurant, bitch, hear, book, men, femi...","[real bitch, steno pad, real bitch”, agg, sex,..."
1,The Sleazy Case Against Mueller’s Probe“Well b...,"{sign, manafort, trump, times, sleazy, Kremlin...",sleazy case mueller probe well public knowledg...,"[trump, russian, steele, campaign, manafort, c...","[steele dossier, hillary clinton, steele, clin..."
2,Fate of Stateless Rohingya Muslims Is in Antag...,"{myanmar, stateless, including, Rohingya, musl...",fate stateless rohingya muslim antagonistic ha...,"[rohingya, myanmar, group, refugee, bangladesh...","[rohingya village, rohingya refugees, rohingya..."
3,"The War of Independence, Seen Through Six Sets...","{revolution, death, eyes, smith, yorks, washin...",war independence seen six set eyesanother shor...,"[washington, shorto, new, smith, coghlan, geor...","[new york, shorto writes, washington, smith, s..."
4,Exquisite GemAdvertisement\n\nWordplay Exquisi...,"{wilber, tough, puzzle, Friday, donaldson, hel...",exquisite gemadvertisement wordplay exquisite ...,"[saturday, puzzle, down, clue, get, wilber, ge...","[brad wilber, well, like, one, get, hard, dona..."


##### KeyBert

In [115]:
kw_model = KeyBERT() #по умолчанию и так для английского

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [118]:
def keybert_for_df(text):
  onegrams = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1))
  bigrams = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2))
  return list(set([elem[0] for elem in onegrams] + [elem[0] for elem in bigrams]))

In [119]:
#попробуем для обработанных и необработанных текстов отдельно

data['KEYBERT_KEYWORDS_PROCESSED'] = data['FULL_TEXT_PROCESSED'].apply(keybert_for_df)
data['KEYBERT_KEYWORDS_UNPROCESSED'] = data['FULL_TEXT'].apply(keybert_for_df)

In [120]:
data

,FULL_TEXT,KEYWORDS_FULL,FULL_TEXT_PROCESSED,TFIDF_KEYWORDS,RAKE_KEYWORDS,KEYBERT_KEYWORDS_PROCESSED,KEYBERT_KEYWORDS_UNPROCESSED
0,"‘I Hear She’s a Real Bitch’: A Swaggering, Fem...","{memoir, shes, sex, swaggering, feminist, men,...",hear real bitch swaggering feminist restaurant...,"[agg, restaurant, bitch, hear, book, men, femi...","[real bitch, steno pad, real bitch”, agg, sex,...","[bitch swaggering, humble, memoir, swaggering,...","[jen agg, bitches, jen, kitchen bitches, bitch..."
1,The Sleazy Case Against Mueller’s Probe“Well b...,"{sign, manafort, trump, times, sleazy, Kremlin...",sleazy case mueller probe well public knowledg...,"[trump, russian, steele, campaign, manafort, c...","[steele dossier, hillary clinton, steele, clin...","[manafort trump, manafort, manafort papadopoul...","[dossier discredited, steele, dossier, mueller..."
2,Fate of Stateless Rohingya Muslims Is in Antag...,"{myanmar, stateless, including, Rohingya, musl...",fate stateless rohingya muslim antagonistic ha...,"[rohingya, myanmar, group, refugee, bangladesh...","[rohingya village, rohingya refugees, rohingya...","[helping rohingya, myanmar, support rohingya, ...","[helping rohingya, disturbing myanmar, myanmar..."
3,"The War of Independence, Seen Through Six Sets...","{revolution, death, eyes, smith, yorks, washin...",war independence seen six set eyesanother shor...,"[washington, shorto, new, smith, coghlan, geor...","[new york, shorto writes, washington, smith, s...","[revolution, separated washington, noble, smit...","[memoir, margaret, independence, margaret cogh..."
4,Exquisite GemAdvertisement\n\nWordplay Exquisi...,"{wilber, tough, puzzle, Friday, donaldson, hel...",exquisite gemadvertisement wordplay exquisite ...,"[saturday, puzzle, down, clue, get, wilber, ge...","[brad wilber, well, like, one, get, hard, dona...","[wilber, exquisite gem, jimmy, donaldson, 49 v...","[saturday crossword, exquisite gemadvertisemen..."


### Шаблоны

Посмотрим на то, что есть в эталоне

In [121]:
for elem in data['KEYWORDS_FULL']:
  print(elem)

{'memoir', 'shes', 'sex', 'swaggering', 'feminist', 'men', 'restaurant', 'book', 'aggs', 'hear', 'real', 'memoirs', 'agg', 'bitch'}
{'sign', 'manafort', 'trump', 'times', 'sleazy', 'Kremlin', 'Trump', 'Manafort', 'papadopoulos', 'Russian', 'russians', 'Mueller', 'probe', 'Clinton', 'steele', 'clinton', 'case', 'campaign', 'muellers', 'russian'}
{'myanmar', 'stateless', 'including', 'Rohingya', 'muslims', 'hands', 'main', 'rohingya', 'Bangladesh', 'group', 'continue', 'antagonistic', 'international', 'refugees', 'reading', 'york', 'violence', 'fate'}
{'revolution', 'death', 'eyes', 'smith', 'yorks', 'washington', 'shortos', 'victory', 'seen', 'sets', 'independence', 'Washington', 'war', 'george', 'coghlan', 'york', 'Shorto', 'New York', 'shorto', 'american'}
{'wilber', 'tough', 'puzzle', 'Friday', 'donaldson', 'help', 'started', 'exquisite', 'Saturday', '_', 'gem', 'thats', 'play', 'rogue', 'im'}


Из-за особенностей разметки синтаксических шаблонов особо нет, но можно выделить другие:
- существительное или глагол(глагольная форма)
- имя собственное в общем, чаще всего фамилии

Это должно быть одно слово

In [132]:
acceptable_tags = {'NN', 'NNP', 'VB', 'VBG', 'VBN'}

def filter_list_of_words(words, tags=acceptable_tags):
  return [word[0] for word in pos_tag(words) if word[1] in tags]

### Считаем метрики

In [159]:
gold = data['KEYWORDS_FULL'].tolist()

def count_metrics(column_name, gold=gold, filter=False):

  list_pred = [set(elem) for elem in data[column_name].tolist()]

  true_positive = 0
  false_positive = 0
  false_negative = 0

  for predicted, real in zip(list_pred, gold):

    if filter == True:
      predicted = {elem.lower() for elem in filter_list_of_words(list(predicted))}
    else:
      predicted = {elem.lower() for elem in predicted}

    real = {elem.lower() for elem in real}

    true_positive += len(real.intersection(predicted))
    false_positive += len(predicted.difference(real)) #то, что встречается в predicted, но не встречается в real
    false_negative += len(real.difference(predicted)) #то, что в real, но не в predicted

  precision = true_positive / (true_positive + false_positive)
  recall = true_positive / (true_positive + false_negative)
  f1 = 2 * (precision * recall) / (precision + recall)

  return precision, recall, f1

In [162]:
from nltk.metrics.scores import precision
column_names = [elem for elem in data.columns if 'FULL' not in elem]

for column in column_names:
  precision_unf, recall_unf, f1_unf = count_metrics(column)
  precision_f, recall_f, f1_f = count_metrics(column, filter=True)

  print(f'Метрики для метода {column.replace("_KEYWORDS", "")}')
  print('Без фильтрации')
  print(f'precision: {precision_unf}')
  print(f'recall: {recall_unf}')
  print(f'F1: {f1_unf}')
  print('С фильтрацией')
  print(f'precision: {precision_f}')
  print(f'recall: {recall_f}')
  print(f'F1: {f1_f}')
  print('\n')

Метрики для метода TFIDF
Без фильтрации
precision: 0.5058823529411764
recall: 0.5375
F1: 0.5212121212121211
С фильтрацией
precision: 0.44642857142857145
recall: 0.3125
F1: 0.36764705882352944


Метрики для метода RAKE
Без фильтрации
precision: 0.34782608695652173
recall: 0.3
F1: 0.3221476510067114
С фильтрацией
precision: 0.36585365853658536
recall: 0.1875
F1: 0.2479338842975207


Метрики для метода KEYBERT_PROCESSED
Без фильтрации
precision: 0.3
recall: 0.1875
F1: 0.23076923076923075
С фильтрацией
precision: 0.3684210526315789
recall: 0.175
F1: 0.23728813559322032


Метрики для метода KEYBERT_UNPROCESSED
Без фильтрации
precision: 0.2
recall: 0.125
F1: 0.15384615384615385
С фильтрацией
precision: 0.18518518518518517
recall: 0.0625
F1: 0.09345794392523364




### Выводы:

- лучше всего справляется TF-IDF, так как обычно в ключевые слова записываются самые частотные в тексте и 1граммы, как в оригинальном тэггинге на категории


In [183]:
print('original', data['KEYWORDS_FULL'][0])
print('bert', data['TFIDF_KEYWORDS'][0])

original {'memoir', 'shes', 'sex', 'swaggering', 'feminist', 'men', 'restaurant', 'book', 'aggs', 'hear', 'real', 'memoirs', 'agg', 'bitch'}
bert ['agg', 'restaurant', 'bitch', 'hear', 'book', 'men', 'feminist', 'real', 'sex', 'memoir', 'woman', 'self', 'culture', 'conference', 'bone', 'bar', 'page']


In [184]:
print('original', data['KEYWORDS_FULL'][1])
print('bert', data['TFIDF_KEYWORDS'][1])

original {'sign', 'manafort', 'trump', 'times', 'sleazy', 'Kremlin', 'Trump', 'Manafort', 'papadopoulos', 'Russian', 'russians', 'Mueller', 'probe', 'Clinton', 'steele', 'clinton', 'case', 'campaign', 'muellers', 'russian'}
bert ['trump', 'russian', 'steele', 'campaign', 'manafort', 'clinton', 'kremlin', 'dossier', 'ukraine', 'papadopoulos', 'mueller', 'newsletter', 'else', 'answer', 'please', 'went', 'vulnerability']


- у RAKE - хуже, скорее всего потому, что а) все  предложения достаточно длинные и контекстное окно большое б) выделять границы по стоп-словам не лучшая идея вообще
- BERT лучше выделяет на обработанных текстах

Посмотрим, что внутри Берта

In [178]:
print('original', data['KEYWORDS_FULL'][0])
print('bert', data['KEYBERT_KEYWORDS_PROCESSED'][0])

original {'memoir', 'shes', 'sex', 'swaggering', 'feminist', 'men', 'restaurant', 'book', 'aggs', 'hear', 'real', 'memoirs', 'agg', 'bitch'}
bert ['bitch swaggering', 'humble', 'memoir', 'swaggering', 'ego agg', 'feminist restaurant', 'writer', 'ego', 'agg memoir', 'swaggering feminist']


In [180]:
print('original', data['KEYWORDS_FULL'][1])
print('bert', data['KEYBERT_KEYWORDS_PROCESSED'][1])

original {'sign', 'manafort', 'trump', 'times', 'sleazy', 'Kremlin', 'Trump', 'Manafort', 'papadopoulos', 'Russian', 'russians', 'Mueller', 'probe', 'Clinton', 'steele', 'clinton', 'case', 'campaign', 'muellers', 'russian'}
bert ['manafort trump', 'manafort', 'manafort papadopoulos', 'mueller', 'putin', 'papadopoulos', 'steele dossier', 'clinton russian', 'manafort indictment', 'russian']


In [181]:
print('original', data['KEYWORDS_FULL'][2])
print('bert', data['KEYBERT_KEYWORDS_PROCESSED'][2])

original {'myanmar', 'stateless', 'including', 'Rohingya', 'muslims', 'hands', 'main', 'rohingya', 'Bangladesh', 'group', 'continue', 'antagonistic', 'international', 'refugees', 'reading', 'york', 'violence', 'fate'}
bert ['revolution', 'separated washington', 'noble', 'smith', 'story revolution', 'independence', 'written washington', 'story smith', '1781', 'independence seen']


Кажется, что Берт выделяет в качестве н-грамм сплошные куски текста, это влияет на осмысленность и на качество 